## Import Libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

## Hyperparameters

In [2]:
RANDOM_SEED = 171

N_FOLDS = 10
N_EPOCHS = 100
BATCH_SIZE = 64

OPTIMIZER = 'adam'
LOSS_FUNCTION = 'mean_squared_error'

HIDDEN_LAYER_UNITS = 256
OUTPUT_LAYER_UNITS = 1
HIDDEN_LAYER_ACTIVATION = 'relu'
OUTPUT_LAYER_ACTIVATION = 'sigmoid'

## Import Data

In [3]:
# Define a dictionary to store all the datasets 
season_statistics = {} 

for year in range(1994, 2024): 
    # Import the CSV file for the corresponding year as a pandas df 
    df = pd.read_csv(f"./data/{year}.csv") 
    
    # Add the dataframe to our dictionary of dataframes 
    season_statistics[year] = df

## Preprocess Data

In [4]:
# Concatenate all season data
df = pd.concat(season_statistics.values(), ignore_index=True)

# Define Categorical and Numerical Columns for Pre-Processing
selected_features = df.columns.drop(['Champion', 'Arena', 'Team', 'Year', 'G'])
# selected_features = ['W%', 'FG%', 'FGA', '3P%', 'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 
#                      'PPG', 'OPPG', 'Age', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'Attend./G']

# Create a Data PreProcessor
preprocessor = ColumnTransformer(transformers = [('num', StandardScaler(), selected_features)])

# Separate the Predictors and Response
X = df[selected_features]
y = df['Champion'].values

# Pre-Process the Input Data
X = preprocessor.fit_transform(X)

## Train Model

In [5]:
# Define the K-Fold Cross Validation Sets
kf = KFold(n_splits = N_FOLDS, shuffle = True, random_state = RANDOM_SEED)

# Create a List for each Model and its Accuracy
model_history, mse_history = [], []

# Perform K-Fold Cross Validation
for train_index, test_index in kf.split(X):
    
    # Split into Training and Testing Datasets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Build the Neural Network Model
    model = Sequential([
        Dense(HIDDEN_LAYER_UNITS, activation = HIDDEN_LAYER_ACTIVATION, input_shape=(X_train.shape[1],)),
        Dense(HIDDEN_LAYER_UNITS, activation = HIDDEN_LAYER_ACTIVATION),
        Dense(HIDDEN_LAYER_UNITS, activation = HIDDEN_LAYER_ACTIVATION),
        Dense(OUTPUT_LAYER_UNITS, activation = OUTPUT_LAYER_ACTIVATION)
    ])
    model.compile(optimizer = OPTIMIZER, loss = LOSS_FUNCTION, metrics = 'mse')
    model.fit(X_train, y_train, epochs = N_EPOCHS, batch_size = BATCH_SIZE, verbose = 1)
    
    # Predict and evaluate
    y_pred = model.predict(X_test).flatten()
    y_pred, y_test = (y_pred * 100), (y_test * 100)
    mse = mean_squared_error(y_test, y_pred)
    
    # Append the Model and MSE to the respective lists
    model_history.append(model)
    mse_history.append(mse)

Epoch 1/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0600 - mse: 0.0600
Epoch 2/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0438 - mse: 0.0438
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0315 - mse: 0.0315
Epoch 4/100
13/13 [==============================] - 0s 1ms/step - loss: 0.0245 - mse: 0.0245
Epoch 5/100
13/13 [==============================] - 0s 1ms/step - loss: 0.0199 - mse: 0.0199
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0169 - mse: 0.0169
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0147 - mse: 0.0147
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0129 - mse: 0.0129
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0105 - mse: 0.0105
Epoch 11/100
13/13 [==============================] - 0s 2m

13/13 [==============================] - 0s 2ms/step - loss: 4.9184e-04 - mse: 4.9184e-04
Epoch 66/100
13/13 [==============================] - 0s 2ms/step - loss: 4.7879e-04 - mse: 4.7879e-04
Epoch 67/100
13/13 [==============================] - 0s 2ms/step - loss: 4.9523e-04 - mse: 4.9523e-04
Epoch 68/100
13/13 [==============================] - 0s 2ms/step - loss: 4.4016e-04 - mse: 4.4016e-04
Epoch 69/100
13/13 [==============================] - 0s 2ms/step - loss: 5.2822e-04 - mse: 5.2822e-04
Epoch 70/100
13/13 [==============================] - 0s 2ms/step - loss: 5.5000e-04 - mse: 5.5000e-04
Epoch 71/100
13/13 [==============================] - 0s 2ms/step - loss: 4.2909e-04 - mse: 4.2909e-04
Epoch 72/100
13/13 [==============================] - 0s 3ms/step - loss: 3.4886e-04 - mse: 3.4886e-04
Epoch 73/100
13/13 [==============================] - 0s 2ms/step - loss: 3.7313e-04 - mse: 3.7313e-04
Epoch 74/100
13/13 [==============================] - 0s 2ms/step - loss: 5.8474e-04 -

13/13 [==============================] - 0s 2ms/step - loss: 0.0021 - mse: 0.0021
Epoch 30/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0021 - mse: 0.0021    
Epoch 31/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0020 - mse: 0.0020    
Epoch 32/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0020 - mse: 0.0020
Epoch 33/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0021 - mse: 0.0021
Epoch 34/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0024 - mse: 0.0024
Epoch 35/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0029 - mse: 0.0029
Epoch 36/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0025 - mse: 0.0025
Epoch 37/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0028 - mse: 0.0028
Epoch 38/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0029 - mse: 0.0029
Epoch 39/100
13/13 [==============================] - 0

13/13 [==============================] - 0s 2ms/step - loss: 3.6364e-04 - mse: 3.6364e-04
Epoch 93/100
13/13 [==============================] - 0s 2ms/step - loss: 4.0787e-04 - mse: 4.0787e-04
Epoch 94/100
13/13 [==============================] - 0s 2ms/step - loss: 3.7667e-04 - mse: 3.7667e-04
Epoch 95/100
13/13 [==============================] - 0s 2ms/step - loss: 3.1418e-04 - mse: 3.1418e-04
Epoch 96/100
13/13 [==============================] - 0s 2ms/step - loss: 4.4161e-04 - mse: 4.4161e-04
Epoch 97/100
13/13 [==============================] - 0s 2ms/step - loss: 4.0116e-04 - mse: 4.0116e-04
Epoch 98/100
13/13 [==============================] - 0s 2ms/step - loss: 3.6132e-04 - mse: 3.6132e-04
Epoch 99/100
13/13 [==============================] - 0s 2ms/step - loss: 3.0362e-04 - mse: 3.0362e-04
Epoch 100/100
3/3 [==============================] - 0s 1ms/step
Epoch 1/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0562 - mse: 0.0562
Epoch 2/100
13/13 [===========

13/13 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012    
Epoch 43/100
13/13 [==============================] - 0s 3ms/step - loss: 0.0012 - mse: 0.0012    
Epoch 44/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012    
Epoch 45/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011    
Epoch 46/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011    
Epoch 47/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011    
Epoch 48/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012    
Epoch 49/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012    
Epoch 50/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011    
Epoch 51/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011
Epoch 52/100
13/13 [=======

13/13 [==============================] - 0s 1ms/step - loss: 0.0184 - mse: 0.0184
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0147 - mse: 0.0147
Epoch 7/100
13/13 [==============================] - 0s 1ms/step - loss: 0.0119 - mse: 0.0119
Epoch 8/100
13/13 [==============================] - 0s 1ms/step - loss: 0.0097 - mse: 0.0097
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0078 - mse: 0.0078
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0074 - mse: 0.0074
Epoch 11/100
13/13 [==============================] - 0s 1ms/step - loss: 0.0056 - mse: 0.0056
Epoch 12/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0045 - mse: 0.0045
Epoch 13/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0039 - mse: 0.0039
Epoch 14/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0049 - mse: 0.0049
Epoch 15/100
13/13 [==============================] - 0s 2ms/step -

13/13 [==============================] - 0s 2ms/step - loss: 6.8245e-04 - mse: 6.8245e-04
Epoch 71/100
13/13 [==============================] - 0s 2ms/step - loss: 6.7324e-04 - mse: 6.7324e-04
Epoch 72/100
13/13 [==============================] - 0s 3ms/step - loss: 6.6301e-04 - mse: 6.6301e-04
Epoch 73/100
13/13 [==============================] - 0s 3ms/step - loss: 6.6534e-04 - mse: 6.6534e-04
Epoch 74/100
13/13 [==============================] - 0s 3ms/step - loss: 7.2977e-04 - mse: 7.2977e-04
Epoch 75/100
13/13 [==============================] - 0s 3ms/step - loss: 7.5160e-04 - mse: 7.5160e-04
Epoch 76/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0010 - mse: 0.0010    
Epoch 77/100
13/13 [==============================] - 0s 3ms/step - loss: 0.0011 - mse: 0.0011
Epoch 78/100
13/13 [==============================] - 0s 2ms/step - loss: 9.3560e-04 - mse: 9.3560e-04
Epoch 79/100
13/13 [==============================] - 0s 3ms/step - loss: 8.3334e-04 - mse: 8.3334

## Evaluate Model

In [6]:
# Print Performance Metrics
print("Average MSE:", np.mean(mse_history), '\n')
for i in range(len(model_history)):
    print(f"Model #{i + 1}, MSE: {mse_history[i]}")
    
# Get the Best Model
best_model_idx = mse_history.index(min(mse_history))
best_model = model_history[best_model_idx]

# Perform Accuracy Test
accuracy = 0
print("\n\nChampionship Prediction (Historic)\n----------------------------------\n")
for year, df in season_statistics.items():
    
    # Get the Actual Champion
    real_champion = df[df['Champion'] == 1]['Team'].values[0]
    
    # Get the Model Prediction
    X = preprocessor.fit_transform(df[selected_features])
    y = best_model.predict(X)
    pred_champion = df.iloc[np.argmax(y)]['Team']
    
    # Evaluate Results
    accuracy += int(real_champion == pred_champion)
    print(f"{year} NBA Season")
    print(f"\tActual Champion: {real_champion}")
    print(f"\tPredicted Champion: {pred_champion}")
    print()
    
print(f"Model Accuracy: {(accuracy / len(season_statistics) * 100):.2f}% ({accuracy}/{len(season_statistics)})")

Average MSE: 356.86586743809505 

Model #1, MSE: 205.64599520881032
Model #2, MSE: 282.4168029564226
Model #3, MSE: 329.61819135837504
Model #4, MSE: 328.5099422388651
Model #5, MSE: 313.55654899903783
Model #6, MSE: 454.8262677038099
Model #7, MSE: 344.18727159246254
Model #8, MSE: 451.7888780379025
Model #9, MSE: 542.6227295619713
Model #10, MSE: 315.48604672329316


Championship Prediction (Historic)
----------------------------------

1/1 [==============================] - 0s 13ms/step
1994 NBA Season
	Actual Champion: Houston Rockets
	Predicted Champion: Houston Rockets

1/1 [==============================] - 0s 12ms/step
1995 NBA Season
	Actual Champion: Houston Rockets
	Predicted Champion: Houston Rockets

1/1 [==============================] - 0s 13ms/step
1996 NBA Season
	Actual Champion: Chicago Bulls
	Predicted Champion: Chicago Bulls

1/1 [==============================] - 0s 13ms/step
1997 NBA Season
	Actual Champion: Chicago Bulls
	Predicted Champion: Chicago Bulls

1/1 [

## Make Prediction

In [7]:
# Get the 2024 NBA Season Data
df = pd.read_csv(f"./data/2024.csv")

# Get the Model Prediction
X = preprocessor.fit_transform(df[selected_features])
y = best_model.predict(X)
pred_champion = df.iloc[np.argmax(y)]['Team']

# Evaluate Results
print(f"2024 Predicted Champion: {pred_champion}")

# Print Championship Probabilities Per Team
print("\n\nChampionship Probabilities (By Team)\n-------------------------------------")
for i in range(len(y)):
    print(f"{df.iloc[i]['Team']}: {(y[i][0] * 100):.2f}%")

1/1 [==============================] - 0s 14ms/step
2024 Predicted Champion: Boston Celtics


Championship Probabilities (By Team)
-------------------------------------
Atlanta Hawks: 0.00%
Boston Celtics: 37.86%
Brooklyn Nets: 0.00%
Charlotte Hornets: 0.00%
Chicago Bulls: 0.12%
Cleveland Cavaliers: 1.43%
Dallas Mavericks: 2.26%
Denver Nuggets: 8.50%
Detroit Pistons: 0.00%
Golden State Warriors: 7.29%
Houston Rockets: 0.22%
Indiana Pacers: 0.06%
Los Angeles Clippers: 18.06%
Los Angeles Lakers: 0.01%
Memphis Grizzlies: 0.00%
Miami Heat: 5.19%
Milwaukee Bucks: 5.93%
Minnesota Timberwolves: 27.63%
New Orleans Pelicans: 2.07%
New York Knicks: 13.91%
Oklahoma City Thunder: 0.16%
Orlando Magic: 0.15%
Philadelphia 76ers: 0.72%
Phoenix Suns: 19.42%
Portland Trail Blazers: 0.00%
Sacramento Kings: 0.01%
San Antonio Spurs: 0.00%
Toronto Raptors: 0.00%
Utah Jazz: 0.00%
Washington Wizards: 0.00%


## Save The Model

In [18]:
# Save the Model
path = './model/DNN_Model.keras'
best_model.save(path)